In [1]:
'065676'

'065676'

# Imports

In [1]:
import backtrader
from helpers.investing import *
from helpers.journal_handler import *

# Defaults

In [2]:
pd.set_option('display.max_rows', 100)

private_key = join(expanduser('~'),'Documents','client_secret.json')
journal = JournalHandler(private_key).get_journal()

In = Investing()

# Work

In [3]:
df = In.show_full_stats(10000,100.10, diff = None, nifty='nifty_50')

df = df[(df['Expected Change %'] <=10) & (df['RSI']<70) & (df['Ichi']>=1)]
df

/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 151.35 for you to afford HDFC
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 162.11 for you to afford BRITANNIA
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 124.23 for you to afford ASIANPAINT
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 144.31 for you to afford TCS
  warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
/home/admin1/Desktop/ML Study/Finance/helpers/investing.py:226: UserWarning: Risk should be atleast 141.76 for you to afford TITAN
  warnings.warn(f"Risk should be atleast {r} for yo

,DATE,OPEN,HIGH,LOW,LTP,CLOSE,52W H,52W L,SYMBOL,Diff,Rising,Ichi,RSI,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
13,2021-08-03,207.50,210.90,206.45,209.75,209.90,239.20,163.35,ITC,0.17,False,1,57.509578,3.871429,Unknown,Unknown,Unknown,Nifty 50,6.45,95.54
8,2021-08-03,434.95,447.90,433.60,446.00,446.50,447.90,175.50,SBIN,6.02,True,2,62.602390,9.117857,Unknown,Unknown,Bullish,Nifty 50,8.46,95.02
14,2021-08-03,987.00,1025.00,986.65,1022.45,1022.05,1119.50,483.25,INDUSINDBK,8.06,True,1,54.987966,24.682143,Unknown,Bullish Engulfing,Inverted Hammer,Nifty 50,8.98,92.37
2,2021-08-03,760.00,770.75,759.55,766.80,767.65,788.15,427.70,TATACONSUM,12.44,True,3,57.378957,15.592857,Unknown,Unknown,Inverted Hammer,Nifty 50,4.96,95.95
0,2021-08-03,568.00,582.00,562.10,578.35,580.20,623.00,394.00,BHARTIARTL,24.00,True,2,68.653284,15.700000,Unknown,Unknown,Unknown,Nifty 50,8.83,77.31
7,2021-08-03,683.00,691.90,676.65,690.05,690.40,692.50,333.75,ICICIBANK,25.79,True,2,65.954821,14.532143,Unknown,Unknown,Unknown,Nifty 50,5.69,98.83
6,2021-08-03,1575.00,1599.90,1544.50,1592.00,1592.40,1614.20,610.70,GRASIM,28.51,True,2,64.232655,39.217857,Reverse V Pattern,Unknown,Hammer,Nifty 50,8.20,65.78
11,2021-08-03,1030.00,1042.95,1024.00,1039.00,1039.45,1067.00,677.05,HCLTECH,39.71,True,2,64.822680,28.650000,Unknown,Unknown,Unknown,Nifty 50,5.11,80.15
10,2021-08-03,1625.00,1638.70,1610.30,1631.00,1634.35,1647.15,843.00,LT,72.93,True,2,65.852602,39.925000,Unknown,Unknown,Hammer,Nifty 50,5.76,94.69
